<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/ejercicio_intro_a_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios Scraping I

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import re

## 1) Titulos de noticias
Descargar la pagina principal de noticias de Tecnología. Utilizar Regex para obtener los titulos de las noticias.

Ayuda:
- los titulos estan encerrados por el tag `h2`
- No tiene que contenter el nombre de otra categoria

In [8]:
infobae_url = 'https://www.infobae.com/tag/tecnologia/'
# Solución
response = requests.get(infobae_url).text

patron = r'<h2.*?>([^<>]+)</h2>'

titulos = re.findall(patron, response)

print(f'En total son {len(titulos)} titulos:\n{titulos}')


En total son 10 titulos:
['Baidu presenta unas gafas inteligentes diseñadas para funcionar como un asistente privado, impulsadas por la IA de ERNIE', 'Logitech lanza nuevas herramientas para gestionar los espacios en el entorno laboral con nuevas herramientas', 'Amazon sorprende con entregas usando drones: la compra llega directo a la puerta en perfecto estado', 'PONANT recibe el apoyo de la CE para su innovador proyecto de descarbonización &#x27;Swap2Zero&#x27;', 'Una argentina hizo historia: fue reconocida por sus conocimientos en IA y ya llegó a la NASA', 'Lista de las mejores bromas para hacer por Whatsapp en el Día de los Santos Inocentes', 'La sorprendente lentitud del cerebro humano desafía las expectativas tecnológicas', 'WhatsApp: cómo activar el modo fin de año para recibir 2025 con la mejor actitud', 'Por qué no cargar los controles del PlayStation con el cargador del celular', 'TikTok integra su inicio de sesión con Lemon8 para facilitar los contenidos fotográficos']


## 2) Spinetta

Descargar letras de canciones

Utilizando beautiful soup descargar todas las canciones de [Spinetta](https://es.wikipedia.org/wiki/Luis_Alberto_Spinetta) que hay en [letras.com](https://www.letras.com/spinetta/)

In [9]:
letras_url = "https://www.letras.com"

def descargar_letras(artista):
    # Solución
    dir_path = '/content/letras'
    path = os.path.join(dir_path, artista)

    if not os.path.exists(dir_path):
      os.mkdir(dir_path)
    if not os.path.exists(path):

      os.mkdir(path)

    os.chdir(path)

    html = requests.get(letras_url + '/' + artista).text
    soup = BeautifulSoup(html, 'html.parser')
    canciones = soup.find_all('a', class_ = 'songList-table-songName')

    # Se realizo un slice a la lista de canciones para no generar demasiados archivos
    # El objetivo es comprobar que funciona y descarga las letras de las 5 primeras canciones
    for cancion in canciones[:5]:
      titulo = cancion.span.text
      url = letras_url + cancion['href']
      song_html = requests.get(url).text
      song_soup = BeautifulSoup(song_html)

      letra_html = song_soup.find('div', attrs={'class': 'lyric-original'})
      letra = ''

      for parrafo in letra_html.find_all('p'):
        texto = parrafo.get_text()
        # Agrega un salto de linea por cada letra mayuscula
        letra += re.sub(r'([A-Z])', '\n\g<1>', texto)
        # Agrega un salto de linea entre parrafos
        letra += '\n'

      with open(f'{titulo}.txt', 'w') as file:
        file.write(letra)

    os.chdir('/content')

    return

artista = "luis-alberto-spinetta"
descargar_letras(artista)

## 3) Datos de la Formula 1

### 3.1) Obtener todos los campeones del mundo
Utilizando beautiful soup se pide descargar los siguientes datos:
- Año del campeonato
- Nombre del piloto
- Nombre del equipo
- Cantidad de carreras
- Cantidad de pilotos

In [11]:
f1_piloto_url = 'https://pitwall.app/seasons?view=grid'
# Traemos el contenido html de la url
response = requests.get(f1_piloto_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Si inspeccionamos el elemento, veremos que los datos están en una tabla llamada data-table
table = soup.find('table', class_='data-table')

# Extraemos las filas de la tabla
rows = table.find_all('tr')

# Lista vacía para almacenar los datos
data = []

# Iteramos con un bucle for sobre las filas para extraer los datos
for row in rows[1:]:  # omitimos el encabezado
    cols = row.find_all('td')
    if cols:
        season = cols[0].text.strip()
        champion = cols[1].text.strip()
        constructor = cols[2].text.strip()
        races = int(cols[3].text.strip())
        drivers = int(cols[4].text.strip())

        data.append({
            'Año': season,
            'Piloto': champion,
            'Equipo': constructor,
            'Carreras': races,
            'nª pilotos': drivers
        })


# Con el objetivo de visualizar la informacion se utiliza pandas
import pandas as pd
df = pd.DataFrame(data)
df



,Año,Piloto,Equipo,Carreras,nª pilotos
0,2024,Max Verstappen,McLaren,24,24
1,2023,Max Verstappen,Red Bull,22,22
2,2022,Max Verstappen,Red Bull,22,22
3,2021,Max Verstappen,Mercedes,22,21
4,2020,Lewis Hamilton,Mercedes,17,23
...,...,...,...,...,...
70,1954,Juan Manuel Fangio,,9,97
71,1953,Alberto Ascari,,9,110
72,1952,Alberto Ascari,,8,108
73,1951,Juan Manuel Fangio,,8,85


### 3.2) Circuitos historicos
Utilizando beautiful soup se pide descargar los siguientes datos:
- Nombre del circuito
- Ubicacion
- Cantidad de carreras
- Link a la pagina del circuito

Ingresando a la pagina del circuito se debe obtener:
- El piloto con más victorias
- Cantidad de victorias
- El piloto con más Pole positions (piloto mas rapido en la clasificación)
- Cantidad de Pole positions

Nota: es recomendable crear una funcion para scrapear la pagina del circuito y devolver los valores pedidos

In [13]:
f1_circuito_url = 'https://pitwall.app/circuits'
response = requests.get(f1_circuito_url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', class_='data-table')

rows = table.find_all('tr')

data = []

for row in rows[1:]:
    cols = row.find_all('td')
    if cols:
        circuito = cols[0].text.strip()
        ubicacion = cols[1].text.strip()
        carreras = cols[3].text.strip()

        # extraemos los enlaces del tag <a> de la primera columna
        enlace_tag = cols[0].find('a')
        enlace = enlace_tag['href'] if enlace_tag else None

        # Construimos URL completa si el href es relativo
        if enlace and enlace.startswith('/'):
            enlace = f'https://pitwall.app{enlace}'

        data.append({
            'circuito': circuito,
            'ubicacion': ubicacion,
            'carreras': carreras,
            'enlace': enlace
        })

df = pd.DataFrame(data)
df


,circuito,ubicacion,carreras,enlace
0,Adelaide Street Circuit,"Adelaide, Australia",11,https://pitwall.app/circuits/adelaide-street-c...
1,Ain-Diab Circuit,"Casablanca, Morocco",1,https://pitwall.app/circuits/ain-diab-circuit
2,Aintree Motor Racing Circuit,"Aintree, United Kingdom",5,https://pitwall.app/circuits/aintree-motor-rac...
3,Algarve International Circuit,"Portimão, Portugal",2,https://pitwall.app/circuits/algarve-internati...
4,Anderstorp Raceway,"Anderstorp, Sweden",6,https://pitwall.app/circuits/anderstorp-raceway
...,...,...,...,...
72,Suzuka Circuit,"Suzuka, Japan",34,https://pitwall.app/circuits/suzuka-circuit
73,Valencia Street Circuit,"Valencia, Spain",5,https://pitwall.app/circuits/valencia-street-c...
74,Watkins Glen International,"Watkins Glen, United States of America",20,https://pitwall.app/circuits/watkins-glen-inte...
75,Yas Marina Circuit,"Abu Dhabi, United Arab Emirates",16,https://pitwall.app/circuits/yas-marina-circuit
